In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Conv1D, MaxPooling1D, Input, Average
import tensorflow as tf

2022-04-19 13:18:47.066580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 13:18:47.066616: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

In [3]:
max_review_length = 500
top_words = 10000
X_train = data[:]
y_train = targets[:]
X_test = data[:500]
y_test = targets[:500]

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [4]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

2022-04-19 13:18:54.739778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-19 13:18:54.740033: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-19 13:18:54.740186: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (macbook-pro): /proc/driver/nvidia/version does not exist
2022-04-19 13:18:54.743637: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


2022-04-19 13:18:55.415552: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 100000000 exceeds 10% of free system memory.


Epoch 1/3
172/782 [=====>........................] - ETA: 5:10 - loss: 0.6191 - accuracy: 0.6483

KeyboardInterrupt: 

### Добавим сверточные слои в модель

In [ ]:
model1 = Sequential()
model1.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model1.summary())

model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

scores1 = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores1[1]*100)) 

### Добавим слои dropout

In [ ]:

model2 = Sequential()
model2.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model2.add(Dropout(0.20))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.20))
model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(64))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model2.summary())

model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=128)

scores1 = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores1[1]*100)) 

### Проведём ансамблирование с помощью слоя Average()

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier

inputs = Input(shape = (max_review_length,))
m1 = model(inputs)
m2 = model1(inputs)
m3 = model2(inputs)
outputs = Average()([m1,m2,m3])

ensemble = tf.keras.Model(inputs = inputs, outputs= outputs)
ensemble.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(ensemble.summary())
ensemble.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=128)

### Введем пользовательский текст

In [ ]:
predict = input()
word_index_org = imdb.get_word_index()
word_index = dict()

for word,number in word_index_org.items():
    word_index[word] = number + 3
string = predict.lower().split(' ')
def parseNumber(str):
    arr = []
    for word in str:
        arr.append(word_index[word])
    result = []
    result.append(arr)
    result =  sequence.pad_sequences(result, maxlen=max_review_length)
    return result
prediction = ensemble.predict(parseNumber(string))
print('Positive:' if np.mean(prediction)>0.6 else 'Negative:',np.mean(prediction))